<a href="https://colab.research.google.com/github/albert-yue/gcn-explanability/blob/master/gcn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%cd gdrive/My\ Drive/gcn_explainability

/content/gdrive/My Drive/gcn_explainability


In [0]:
! git config --global user.email "ecrainbow8@gmail.com"

[master bbef074] Update


In [0]:
%ls

data/  gcn.ipynb  logs.pt  notebooks/  README.md  src/


In [0]:
from src.models.gcn import GCNTreebankModel
import torch
import numpy as np
import scipy.sparse as sparse
import scipy
from src.data import Corpus, get_data, get_labels, get_vocabulary
from src.utils import load_sparse_tensor
import torch.nn as nn
from torch.autograd import Variable
from src.preprocessing import build_adj_matrix, normalize_adj
from src.stanford_preprocessing import load_glove_embeddings

logs = (200, 0.5, 41664, 23, 'logs.pt')
ng20 = (200, 0.5, 80035, 20, 'gcn_20ng_full.pt', 61189, 18846, 11314,7532)
trees = ('sst_train_tune_hidden_30_lr_0.00015.pt', 200, 30 ,5, 0.5)
path, input_size, hidden_size, output_size, dropout = trees

model = GCNTreebankModel(input_size, hidden_size, output_size, dropout)


model.load_state_dict(torch.load(path))
print("hi")
test_data = torch.load("data/stanford_test.pt")
inputs = test_data['inputs']
print(inputs[0].label)
embeddings = torch.load("data/embeddings.pt")
print(embeddings)

hi
2


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [0]:
# reverse_map = {}
# for word, embedding in embeddings.items():
#   reverse_map[embedding] = word

# print(len(embeddings.items()))

401784


In [0]:
treebank = test_data['treebank']
def getLeft(node):
  if node.children:
    return getLeft(node.children[0])
  else:
    return node.text
def getRight(node):
  if node.children:
    return getRight(node.children[-1])
  else:
    return node.text

def node_to_words(node, words=[]):
  if node.children:
    node_to_words(node.children[0], words)
    words.append(getLeft(node) + " --- " + getRight(node))
    if (len(node.children) == 2):
      node_to_words(node.children[1], words)
    return words
  else:
    words.append(node.text)
    return words



In [0]:
def matrices(path, adj, inp, label):
  model = GCNTreebankModel(input_size, hidden_size, output_size, dropout)
  model.load_state_dict(torch.load(path))
  model.eval()
  loss_fn = nn.CrossEntropyLoss()
  model.zero_grad()
  output = model(adj, inp)
  loss = loss_fn(output, label)
  loss.retain_grad()
  loss.backward()
  weight_matrices = []
  gradients = []
  for n, p in model.named_parameters():
    print(n)
    weight_matrices.append(p.detach().numpy())
    gradients.append(p.grad.numpy())
  return weight_matrices, gradients, output

In [0]:
def grad_cam_avg(adj, inp, weight_matrices, gradients):
  V = np.matmul(adj, inp)
  F_1 = scipy.special.expit(np.matmul(V, weight_matrices[0]))
  F_2 = scipy.special.expit(np.matmul(V, np.matmul(inp.T, (np.matmul(F_1, weight_matrices[1])))))
  F = [F_1, F_2]
  grad_cams = []
  for l in range(2):
      grad_cam = []
      grad_cams.append(grad_cam)
      alphas = np.mean(gradients[l], axis = 0)
      for n in range(adj.shape[0]):
        grad_cam.append(np.maximum(0, np.dot(alphas, F[l][n, :])))

  grad_cam_avgs = np.mean(grad_cams, axis=0)
  return grad_cams, grad_cam_avgs

In [0]:
def convert_to_words(top_words, sentence_words):
  words = []
  for embedding in top_words:
      words.append(sentence_words[embedding])
  return words

In [0]:
indices = [6, 493]
for i in indices:
  single_corpus = inputs[i]
  sentence = treebank[i].text
  node_words = node_to_words(treebank[i].tree, [])
  adj = single_corpus.adj
  label = torch.LongTensor([single_corpus.label])
  inp = single_corpus.inp
  weight_matrices, gradients, output = matrices(path, adj, inp, label)
  grad_cams, grad_cam_avgs = grad_cam_avg(adj, inp, weight_matrices, gradients)
  print(sentence)
  print(label)
  print(np.argmax(output.detach().numpy()))
  for name, grad_cam_vals in [('layer1', grad_cams[0]), ('layer2', grad_cams[1]), ('overall', grad_cam_avgs)]:
    print(name+":\n")
    top_words_idx = np.asarray(grad_cam_vals).argsort()[-10:]
    top_words = convert_to_words(reversed(top_words_idx), node_words)
    for word in top_words:
      print(word)
  
  
  max_grads = np.asarray(np.linalg.norm(np.matmul(inp,np.matmul(gradients[0], gradients[1])),axis=1)).argsort()[-10:]
  print(convert_to_words(reversed(max_grads), node_words))



layer1.weight
layer2.weight
['Steers', 'turns', 'in', 'a', 'snappy', 'screenplay', 'that', 'curls', 'at', 'the', 'edges', ';', 'it', "'s", 'so', 'clever', 'you', 'want', 'to', 'hate', 'it', '.']
tensor([3])
3
layer1:

.
screenplay
the
at --- edges
at
curls --- edges
curls
that --- edges
that
a --- edges
layer2:

Steers
screenplay
curls
snappy
the
at --- edges
at
curls --- edges
that --- edges
that
overall:

Steers
screenplay
curls
snappy
the
at --- edges
at
curls --- edges
that --- edges
that
['it --- it', 'you --- it', 'so --- it', 'to --- it', "'s --- it", 'want --- it', 'that --- edges', 'a --- edges', 'the --- edges', 'in --- edges']
layer1.weight
layer2.weight
['This', 'boisterous', 'comedy', 'serves', 'up', 'a', 'cruel', 'reminder', 'of', 'the', 'fate', 'of', 'hundreds', 'of', 'thousands', 'of', 'Chinese', ',', 'one', 'which', 'can', 'only', 'qualify', 'as', 'a', 'terrible', 'tragedy', '.']
tensor([2])
3
layer1:

a --- tragedy
the --- tragedy
hundreds --- tragedy
Chinese --- trag

In [0]:


total = 0
max_num = 10
for i in range(1, 7500):
  model.eval()
  model.zero_grad()
  output = model(inputs[i].adj, inputs[i].inp)
  if inputs[i].label == np.argmax(output.detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

total = 0
for i in range(1, 1000):
  model.eval()
  model.zero_grad()
  output = model(inputs[i].adj, inputs[i].inp)
  if inputs[i].label == np.argmax(output.detach().numpy()):
    total += 1
print(total/1000)

total = 0
for i in range(500, 1, -1):
  model.eval()
  model.zero_grad()
  output = model(inputs[i].adj, inputs[i].inp)
  if inputs[i].label != np.argmax(output.detach().numpy()):
    print(i)
    total += 1
    if total == max_num:
      break

4
6
8
9
10
11
12
14
17
19
0.424
500
499
498
497
493
491
490
489
486
482


In [0]:
def p_carrot_val(carrot, p, weight_matrix):
  p_carrot = np.matmul(np.maximum(0, weight_matrix), carrot.T)
  p_carrot = p_carrot / np.sum(p_carrot.numpy(), axis= 0)
  p_carrot = np.matmul(p_carrot, p)
  return p_carrot

def p_F_val(v, F, p_carrot):
  p_F = np.matmul(v, F)
  p_F = p_F / np.sum(p_F.numpy(), axis= 0)
  p_F = np.matmul(p_F, p_carrot.T)
  return p_F

def EB(adj, inp, weight_matrices, gradients, output):
  V = np.matmul(adj, inp)
  V_ = np.matmul(inp.T, V)
  F_1 = scipy.special.expit(np.matmul(V_, weight_matrices[0]))
  F_2 = scipy.special.expit(np.matmul(V_, np.matmul(F_1, weight_matrices[1])))
  F = [F_1, F_2]
  F_carrot = [np.matmul(V_, F_1), np.matmul(V_, F_2)]
  p_F3 = np.tile(output.detach().numpy(), (inp.shape[1],1))
  p_carrot_F2 = p_carrot_val(F_carrot[1], p_F3, weight_matrices[1])
  p_F2 = p_F_val(V_, F[1], p_carrot_F2)
  p_carrot_F1 = p_carrot_val(F_carrot[0], p_F2, weight_matrices[0])
  p_F1 = p_F_val(V, F[0], p_carrot_F1)
  p_F = [p_F1, p_F2, p_F3]
  p_carrot_F = [p_carrot_F1, p_carrot_F2]
  eb_vals = np.mean(p_F1.numpy(), axis=1)
  return eb_vals


for i in indices:
  single_corpus = inputs[i]
  sentence = treebank[i].text
  node_words = node_to_words(treebank[i].tree, [])
  adj = single_corpus.adj
  label = torch.LongTensor([single_corpus.label])
  inp = single_corpus.inp
  weight_matrices, gradients, output = matrices(path, adj, inp, label)
  eb_vals = EB(adj, inp, weight_matrices, gradients, output)


  print(sentence)
  print(label)
  print(np.argmax(output.detach().numpy()))
  top_words_idx = np.asarray(grad_cam_vals).argsort()[-10:]
  top_words = convert_to_words(reversed(top_words_idx), node_words)
  for word in top_words:
    print(word)



layer1.weight
layer2.weight
['Steers', 'turns', 'in', 'a', 'snappy', 'screenplay', 'that', 'curls', 'at', 'the', 'edges', ';', 'it', "'s", 'so', 'clever', 'you', 'want', 'to', 'hate', 'it', '.']
tensor([3])
3
curls --- edges
Steers --- ;
so --- it
want --- it
clever --- it
Steers --- it
at
clever
;
that --- edges
layer1.weight
layer2.weight
['This', 'boisterous', 'comedy', 'serves', 'up', 'a', 'cruel', 'reminder', 'of', 'the', 'fate', 'of', 'hundreds', 'of', 'thousands', 'of', 'Chinese', ',', 'one', 'which', 'can', 'only', 'qualify', 'as', 'a', 'terrible', 'tragedy', '.']
tensor([2])
3
a --- tragedy
the --- tragedy
hundreds --- tragedy
Chinese --- tragedy
of --- tragedy
of --- tragedy
of
of
of
cruel --- reminder
